In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.0.1
0.15.2


# Setting up device-agnostic code

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'